In [4]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Set a seed for reproducibility
manualSeed = 65536
#manualSeed = random.randInt(1, 999999999) <--- For new results
print("DCGAN Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.use_deterministic_algorithms(True) # For reproducible results

DCGAN Seed:  65536


In [5]:
# Root directory
dataroot = "data/celeba"
# Number of workers for dataloader
workers = 2
# Training batch size
batch_size = 128
# Spatial size of training images. All must be resized using transformer.
image_size = 64
# Number of channels in training images. 3 = color.
nc = 3
# Size of feature maps in generator
ngf = 64
# Size of feature maps in discriminator
ndf = 64
# Number of epochs
num_epochs = 5
# Beta1 hyperparameter for Adam optimizers
beta1 = 0.5
# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1